In [8]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt

import os
import sys

from IPython.display import display, Image

from scipy import ndimage

import random
import cv2 
from sklearn.model_selection import train_test_split
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
%matplotlib inline

def show_img(img):
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    if len(img.shape) == 3:
        plt.imshow(img)
    else:
        img2 = img
        plt.imshow(img2, cmap='gray')
        plt.show()

# Get three channel data

In [2]:
%load_ext autoreload
%autoreload 2
from utils.preprocess_utils import *

In [3]:
shape = (64, 64)
channel = 3
data = generate_data_set_for_training(shape, channel)

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)


In [4]:
X_train = data['X_train']
X_val = data['X_val']
y_train = data['y_train']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']

# Try pretrianed Vgg

In [5]:
import numpy as np
import os
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import regularizers
import keras.utils as ku
from keras import backend as K
from keras import optimizers
import tensorflow as tf
import pickle
import matplotlib.pyplot as ply

Using TensorFlow backend.


In [6]:
from utils.vgg16_utils import *

In [ ]:
# from scratch
run_pretrained_vgg16()

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)
Train on 69076 samples, validate on 17269 samples
Epoch 1/50


In [7]:
_, row, col, channel = X_train.shape
print('chanel', channel)
print('row', row)
print('col', col)

chanel 3
row 64
col 64


In [9]:
# Params
digLen = 5
numDigits = 11

In [10]:
preTrainModel = VGG16(include_top = False, weights = 'imagenet')
preTrainModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [11]:
ptInput = keras.Input(shape = (row,col,channel), name  = 'inputVGGPreTrain')
pt_vgg16 = preTrainModel(ptInput)

In [12]:
Mout = Flatten(name = 'flatten')(pt_vgg16)
Mout = Dense(1024, activation='relu', name = 'FC1_4096')(Mout)
Mout = Dense(1024, activation='relu', name = 'FC1_512')(Mout)

In [13]:
numd_SM = Dense(digLen,    activation='softmax',name = 'num')(Mout)
dig1_SM = Dense(numDigits, activation='softmax',name = 'dig1')(Mout)
dig2_SM = Dense(numDigits, activation='softmax',name = 'dig2')(Mout)
dig3_SM = Dense(numDigits, activation='softmax',name = 'dig3')(Mout)
dig4_SM = Dense(numDigits, activation='softmax',name = 'dig4')(Mout)
numB_SM = Dense(2, activation='softmax',name = 'nC')(Mout)
out = [numd_SM, dig1_SM ,dig2_SM, dig3_SM, dig4_SM,numB_SM] #numd_SM

In [14]:
    epochs = 50
    batch_size = 64
    lr = 0.001
    beta_1 = 0.9
    beta_2 = 0.999
    epsilon = None
    decay = 0.0
    amsgrad = True
    # defind optimizer



    optim = optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay, amsgrad=amsgrad)

In [15]:
vggPreTrain = keras.Model(inputs = ptInput, outputs = out)

vggPreTrain.compile(loss = 'sparse_categorical_crossentropy', #ceLoss ,
                    optimizer= optim,
                    metrics=  ['accuracy']) #[])
vggPreTrain.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputVGGPreTrain (InputLayer)   (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    inputVGGPreTrain[0][0]           
__________________________________________________________________________________________________
flatten (Flatten)               (None, 2048)         0           vgg16[1][0]                      
__________________________________________________________________________________________________
FC1_4096 (Dense)                (None, 1024)         2098176     flatten[0][0]                    
__________________________________________________________________________________________________
FC1_512 (D

In [16]:
callback = []
checkpointer = keras.callbacks.ModelCheckpoint(filepath='saved_models/VGGPreTrained_classifier_corrected.hdf5',
                                               monitor='loss',
                                               save_best_only=True,
                                               verbose=2)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 4,
                                              cooldown= 1,
                                              min_lr = 0.0001)
es = keras.callbacks.EarlyStopping(monitor= 'loss',
                                   min_delta=0.000001,
                                   patience=5,
                                   verbose=1,
                                   mode='auto')
callback.append(es)
callback.append(checkpointer)
callback.append(reduce_lr)

In [16]:
%%time
vggHistory = vggPreTrain.fit(x = X_train,
                             y = list(y_train.T),
                             batch_size = batch_size,
                             epochs=epochs,
                             verbose=1,
                             shuffle = True,
                             validation_data = (X_val, list(y_val.T)),
                             callbacks= callback)

print(vggHistory.history.keys())
modName = 'vgg16_PreTrain_corrected'
# list all data in history
print(vggHistory.history.keys())

Train on 69076 samples, validate on 17269 samples
Epoch 1/50
69076/69076 [==============================] - 88s 1ms/step - loss: 2.3541 - num_loss: 0.2782 - dig1_loss: 0.7762 - dig2_loss: 0.8120 - dig3_loss: 0.3901 - dig4_loss: 0.0851 - nC_loss: 0.0125 - num_acc: 0.8871 - dig1_acc: 0.7272 - dig2_acc: 0.7126 - dig3_acc: 0.8973 - dig4_acc: 0.9849 - nC_acc: 0.9968 - val_loss: 2.1584 - val_num_loss: 0.2337 - val_dig1_loss: 0.6788 - val_dig2_loss: 0.8026 - val_dig3_loss: 0.3728 - val_dig4_loss: 0.0702 - val_nC_loss: 3.9290e-04 - val_num_acc: 0.9056 - val_dig1_acc: 0.7723 - val_dig2_acc: 0.7128 - val_dig3_acc: 0.8953 - val_dig4_acc: 0.9865 - val_nC_acc: 0.9999

Epoch 00001: loss improved from inf to 2.35407, saving model to saved_models/VGGPreTrained.classifier.hdf5
Epoch 2/50
69076/69076 [==============================] - 80s 1ms/step - loss: 2.1642 - num_loss: 0.2668 - dig1_loss: 0.6669 - dig2_loss: 0.7713 - dig3_loss: 0.3691 - dig4_loss: 0.0824 - nC_loss: 0.0076 - num_acc: 0.8998 - dig1_a


Epoch 00012: loss improved from 0.44216 to 0.35039, saving model to saved_models/VGGPreTrained.classifier.hdf5
Epoch 13/50
69076/69076 [==============================] - 80s 1ms/step - loss: 0.2932 - num_loss: 0.0226 - dig1_loss: 0.0756 - dig2_loss: 0.0985 - dig3_loss: 0.0697 - dig4_loss: 0.0267 - nC_loss: 1.5327e-05 - num_acc: 0.9923 - dig1_acc: 0.9762 - dig2_acc: 0.9685 - dig3_acc: 0.9764 - dig4_acc: 0.9909 - nC_acc: 1.0000 - val_loss: 0.5383 - val_num_loss: 0.0562 - val_dig1_loss: 0.1336 - val_dig2_loss: 0.1956 - val_dig3_loss: 0.1193 - val_dig4_loss: 0.0336 - val_nC_loss: 1.2015e-05 - val_num_acc: 0.9844 - val_dig1_acc: 0.9625 - val_dig2_acc: 0.9440 - val_dig3_acc: 0.9655 - val_dig4_acc: 0.9899 - val_nC_acc: 1.0000

Epoch 00013: loss improved from 0.35039 to 0.29315, saving model to saved_models/VGGPreTrained.classifier.hdf5
Epoch 14/50
69076/69076 [==============================] - 80s 1ms/step - loss: 0.2656 - num_loss: 0.0196 - dig1_loss: 0.0699 - dig2_loss: 0.0883 - dig3_loss:


Epoch 00035: loss improved from 0.07936 to 0.03852, saving model to saved_models/VGGPreTrained.classifier.hdf5
Epoch 36/50
69076/69076 [==============================] - 80s 1ms/step - loss: 0.0129 - num_loss: 9.3879e-04 - dig1_loss: 0.0033 - dig2_loss: 0.0044 - dig3_loss: 0.0031 - dig4_loss: 0.0012 - nC_loss: 1.7067e-06 - num_acc: 0.9998 - dig1_acc: 0.9993 - dig2_acc: 0.9989 - dig3_acc: 0.9993 - dig4_acc: 0.9998 - nC_acc: 1.0000 - val_loss: 0.6957 - val_num_loss: 0.0732 - val_dig1_loss: 0.1842 - val_dig2_loss: 0.2447 - val_dig3_loss: 0.1579 - val_dig4_loss: 0.0358 - val_nC_loss: 2.1615e-06 - val_num_acc: 0.9863 - val_dig1_acc: 0.9666 - val_dig2_acc: 0.9567 - val_dig3_acc: 0.9744 - val_dig4_acc: 0.9934 - val_nC_acc: 1.0000

Epoch 00036: loss improved from 0.03852 to 0.01288, saving model to saved_models/VGGPreTrained.classifier.hdf5
Epoch 37/50
69076/69076 [==============================] - 80s 1ms/step - loss: 0.0078 - num_loss: 5.7797e-04 - dig1_loss: 0.0021 - dig2_loss: 0.0028 - di

# vgg_pretrain_result

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train loss: 0.0012191442427505605
number_of_digits digit1 digit2 digit3 digit4
Train accuracy: [100.0, 99.99710463836932, 99.99710463836932, 100.0, 100.0, 100.0]
Train sequence accuracy: 99.99420927673867
Validation loss: 0.8632769504238671
number_of_digits digit1 digit2 digit3 digit4
Validation accuracy: [98.65076148010886, 96.69928774103886, 95.89437720771325, 97.56789623023916, 99.35722971799179, 100.0]
Validation sequence accuracy: 92.33887312525334
Test loss: 1.9249597305217272
number_of_digits digit1 digit2 digit3 digit4
Test accuracy: [96.96157967243226, 91.51997550895454, 89.759681616409, 95.83652227154447, 99.44129802540947, 100.0]
Test sequence accuracy: 81.93020052043471


{'train_acc': [100.0,
  99.99710463836932,
  99.99710463836932,
  100.0,
  100.0,
  100.0],
 'test_acc': [96.96157967243226,
  91.51997550895454,
  89.759681616409,
  95.83652227154447,
  99.44129802540947,
  100.0],
 'val_acc': [98.65076148010886,
  96.69928774103886,
  95.89437720771325,
  97.56789623023916,
  99.35722971799179,
  100.0],
 'train_seq_acc': 99.99420927673867,
 'test_seq_acc': 81.93020052043471,
 'val_seq_acc': 92.33887312525334,
 'train_score': [0.0012191442427505605,
  9.542184659218849e-05,
  0.00047577119569368494,
  0.0003629728998552712,
  0.00022080434270599865,
  6.387599376335974e-05,
  2.979689108205491e-07,
  1.0,
  0.9999710463836933,
  0.9999710463836933,
  1.0,
  1.0,
  1.0],
 'test_score': [1.9249597305217272,
  0.22608713149070242,
  0.6083131103722972,
  0.7564571753749809,
  0.2903091489847405,
  0.04378623184903659,
  6.951734232663811e-06,
  0.9696157967243226,
  0.9151997550712981,
  0.89759681616409,
  0.9583652226971974,
  0.9944129802540946,
  1

#  detection